In [1]:
from pgml import Database
import os
import json

In [3]:
local_pgml = "postgres://postgres@127.0.0.1:5433/pgml_development"

conninfo = os.environ.get("PGML_CONNECTION",local_pgml)
db = Database(conninfo,min_connections=4)

In [4]:
collection_name = "test_collection_1"
collection = db.create_or_get_collection(collection_name)

In [9]:
conn = db.pool.getconn()
emb_table = collection._create_or_get_embeddings_table(conn, model_id=1,splitter_id=1)
db.pool.putconn(conn)

missing FROM-clause entry for table "test_collection_1"
LINE 1: ...forms (table_name, task, model, splitter) VALUES (test_colle...
                                                             ^


In [ ]:
from datasets import load_dataset

data = load_dataset("squad", split="train")
data = data.to_pandas()
data.head()

data = data.drop_duplicates(subset=["context"])
print(len(data))
data.head()

documents = [
    {
        'text': r['context'],
        'metadata': {
            'title': r['title']
        }
    } for r in data.to_dict(orient='records')
]
documents[:3]

In [ ]:
collection.upsert_documents(documents[0:200])
collection.generate_chunks()
collection.generate_embeddings()

In [ ]:
results = collection.vector_search("Who won 20 Grammy awards?", top_k=2)
print(json.dumps(results,indent=2))

In [ ]:
collection.register_model(model_name="paraphrase-MiniLM-L6-v2")

In [ ]:
collection.get_models()

In [ ]:
print(json.dumps(collection.get_models(),indent=2))

In [ ]:
collection.generate_embeddings(model_id=2)

In [ ]:
results = collection.vector_search("Who won 20 Grammy awards?", top_k=2, model_id=2)
print(json.dumps(results,indent=2))

In [ ]:
collection.register_model(model_name="hkunlp/instructor-xl", model_params={"instruction": "Represent the Wikipedia document for retrieval: "})

In [ ]:
collection.get_models()

In [ ]:
collection.generate_embeddings(model_id=3)

In [ ]:
results = collection.vector_search("Who won 20 Grammy awards?", top_k=2, model_id=3, query_parameters={"instruction": "Represent the Wikipedia question for retrieving supporting documents: "})
print(json.dumps(results,indent=2))

In [ ]:
collection.register_text_splitter(splitter_name="RecursiveCharacterTextSplitter",splitter_params={"chunk_size": 100,"chunk_overlap": 20})

In [ ]:
collection.generate_chunks(splitter_id=2)

In [ ]:
collection.generate_embeddings(splitter_id=2)

In [ ]:
results = collection.vector_search("Who won 20 Grammy awards?", top_k=2, splitter_id=2)
print(json.dumps(results,indent=2))

In [ ]:
db.delete_collection(collection_name)